In [75]:
import pandas as pd
import collections
from typing import Callable
import datetime

In [94]:
def calculate_mature_shares(path: str, id: str, logger: Callable[[str], None]) -> float:
    report = pd.read_csv(path, skipfooter=1, parse_dates=True, engine='python')
    logger(f"{len(report)=}")
    report = report.drop(columns=['Activity Date', 'Process Date', 'Description'])
    report = report.loc[report['Instrument'] == id]
    print(report['Trans Code'].unique())
    report = report.loc[report['Trans Code'].isin(["Buy", "Sell"])]
    logger(f"{len(report)=}")
    report['Settle Date'] = pd.to_datetime(report['Settle Date'])
    report = report.sort_values(by='Settle Date')
    logger(report.head(20))
    fifo = collections.deque()
    print("===")
    
    class Record:
        def __init__(self, date: datetime.datetime, quantity: float) -> None:
            self.date = date
            self.quantity = quantity


    for idx, record in report.iterrows():
        date, stock_id, code, quantity, price, amount = record
        if code == "Buy":
            fifo.append(Record(date.to_pydatetime(), float(quantity)))
        if code == "Sell":
            sell_quantity = float(quantity)
            while sell_quantity > 0:
                if len(fifo) > 0:
                    earliest_buy = fifo.popleft()
                    earliest_buy.quantity -= sell_quantity
                    if earliest_buy.quantity > 0:
                        fifo.appendleft(earliest_buy)
                    else:
                        sell_quantity = abs(earliest_buy.quantity)
                else:
                    print("Error: not enough stock to sell. Skip")
                    sell_quantity = 0
    print(fifo)

    mature_quantity = 0
    one_year = datetime.timedelta(days=365)
    current_date = datetime.datetime.combine(datetime.date.today(), datetime.datetime.min.time())
    while len(fifo) > 0:
        earliest_buy = fifo.popleft()
        holding_time = current_date - earliest_buy.date
        print(f"Compare {current_date=} and {earliest_buy.date=}, {holding_time=}")
        if holding_time > one_year:
            print("More than a year")
            mature_quantity += earliest_buy.quantity
        else:
            print("Less than a year")

    return mature_quantity

def notebook_print(log: str) -> None:
    print(log)

calculate_mature_shares("./sample.csv", "VOO", notebook_print)

len(report)=5499
['Buy' 'SLIP' 'CDIV' 'Sell']
len(report)=472
     Settle Date Instrument Trans Code Quantity    Price       Amount
5403  2019-11-29        VOO        Buy        2  $288.28    ($576.56)
5352  2020-01-24        VOO        Buy        1  $305.17    ($305.17)
5335  2020-02-07        VOO        Buy        2  $305.16    ($610.32)
5225  2020-03-11        VOO       Sell        5  $254.28    $1,271.38
5177  2020-03-26        VOO        Buy        5  $217.87  ($1,089.35)
5133  2020-04-17        VOO       Sell        5  $255.18    $1,275.87
5021  2020-04-24        VOO        Buy        5  $256.87  ($1,284.35)
5024  2020-04-24        VOO        Buy        5  $256.66  ($1,283.29)
5008  2020-04-27        VOO       Sell       10  $256.90    $2,568.94
4974  2020-04-28        VOO        Buy        4  $258.08  ($1,032.32)
4942  2020-04-28        VOO        Buy       10  $259.34  ($2,593.39)
4900  2020-04-30        VOO        Buy        6  $263.68  ($1,582.08)
4889  2020-04-30        VOO 

0